# Goal

I'm working with non spatially-referenced images produced from a laser ablation system. Along with the image, the software produces an "align" file of the same name - an xml file that contains the XY center point of each image (coordinates are in microns) and the XY size (in microns) along with any rotation. 

I would like to display the images (we typically generate about 1000 images / day) in their correct relative positions and sizes.

The laser has a stage with a cartesian positive XY coordinate system (in microns), and the origin in the SW corner. 

Is there a way to either turn the images into georeferenced geotiffs (using the information in the xml file)? So far I have gotten to the step of generating an affine transform for each scanned image based on it's size and center coordinate in microns and rotation metadata. What I don't have is a definition of a custom CRS in WKT. I don't have any experience with the WKt format but it seems like the only way to represent non-Earth CRSs.

Are there any resources for defining a custom CRS in WKT? I know the units are in microns, the center point of my datum, and the height and width of the coordinate reference system.


### Matching png paths to metadata paths and extracting the metadata into tabular format

In [1]:
import pandas as pd
import skimage as ski
from pathlib import Path
import xml.etree.ElementTree as et 
import numpy as np

def match_png_align(png_file_path, align_file_path):
    """Tests if png and align file IDs are the same."""
    png_lst = str(png_file_path.name).split("_")
    align_lst = str(align_file_path.name).split("_")
    if png_lst[1] == align_lst[1] \
    and png_lst[2] == align_lst[2] \
    and png_lst[3] == align_lst[3] \
    and png_lst[-1][:-4] == align_lst[-1][:-6]:
        return (png_file_path, align_file_path)


def read_transform_inputs_png(meta_path):
    """Reads xml info about the scanned image used to transform coordinates"""
    xtree = et.parse(meta_path)
    root = xtree.getroot()
    center_info = root[0][2].text.split(",")
    size_info = root[0][3].text.split(",")
    extra_info = root[0][0].text.split(";")
    return {"rotation": float(root[0][1].text), 
           "center_x": float(center_info[0]),
           "center_y" : float(center_info[1]),
           "size_x" : float(size_info[0]),
           "size_y" : float(size_info[1]),
           "brightness": float(extra_info[0].split("=")[1]),
           "contrast": float(extra_info[1].split("=")[1]),
           "autoexposure": float(extra_info[2].split("=")[1]),
           "exposuretime": float(extra_info[3].split("=")[1])}

def read_transform_inputs_datum(datum_path):
    xtree = et.parse(datum_path)
    root = xtree.getroot()
    return {
        "rotation" : float(root[0][0].text),
        "center_x" : float(root[0][1].text.split(",")[0]),
        "center_y" : float(root[0][1].text.split(",")[1]),
        "size_x" : float(root[0][2].text.split(",")[0]),
        "size_y" : float(root[0][2].text.split(",")[1]),
        "focus" : float(root[0][3].text)
    }

images_p = Path("images/")

align_paths = list(images_p.glob("ScanImage*.Align"))
png_paths = list(images_p.glob("ScanImage*.png"))

matches = []
for a in align_paths:
    for p in png_paths:
        if match_png_align(p, a):
            matches.append((p,a))

image_df = pd.DataFrame(matches, columns=["png","meta"])


image_df = image_df.join(pd.json_normalize(image_df.meta.apply(read_transform_inputs_png)))

image_df['source_shape'] = image_df.png.apply(lambda x: ski.io.imread(x).shape)

image_df = image_df.join(pd.DataFrame(image_df['source_shape'].tolist(), index=image_df.index, columns=["source_size_y", "source_size_x", "source_size_band"])  )  

datum_im_p = list(images_p.glob("Image_1*"))[0]
datum_align_p = list(images_p.glob("Image_1*"))[1]

# Defining the transform for each png

The format for the transfrm is (uperleftx, scalex, skewx, uperlefty, skewy, scaley) and defines how to map pixel coordinates to CRS coordinates. The only thing missing is a custom CRS that is defined from the datum image metadata.

X and Y pixel resolution of the datum image

In [7]:
image_df['resolution_x'] = image_df['size_x'] / image_df['source_size_x']
image_df['resolution_y'] = image_df['size_y'] / image_df['source_size_y']

Since we know the center and size in projection coordinate of each png scan, we can get the upper left.

In [8]:
image_df['upleftx'] = image_df.center_x - image_df.size_x / 2
image_df['uplefty'] = image_df.center_y - image_df.size_y / 2

In [10]:
import affine


In [11]:
# Specify raster location through geotransform array
# (uperleftx, scalex, skewx, uperlefty, skewy, scaley)
image_df["affine_transform"] = image_df.apply(lambda row: affine.Affine(row['upleftx'], row['resolution_x'], row['rotation'], row['uplefty'], row['rotation'], -row['resolution_y']), axis=1)

# Defining the custom CRS, how do I do this with WKT version 1?

I'm unsure how this rotation should be used (it's only present in the reference datum image). Were all png images captured under this rotation, i.e., are their rotations relative to this already rotated image? Assuming that they are for now. If this is incorrect, the row rotation above needs adjusting.

In [37]:
datum_meta = read_transform_inputs_datum(datum_align_p)
datum_meta

{'rotation': 0.53,
 'center_x': 42085.0,
 'center_y': 47699.0,
 'size_x': 117207.0,
 'size_y': 104307.0,
 'focus': 0.0}

In [36]:
from rasterio.crs import CRS

Rasterio supports defining a CRS from WKT version 1 for versions above 1.0.14

In [ ]:
custom_WKT_string = ????

In [ ]:
CRS.from_wkt(custom_WKT_string)

### After defining the custom crs we can reproject each image and save to a geotiff. Probably need to use calculate_default_transform to ge tthe width and height of the output.

In [ ]:
from rasterio.warp import reproject

In [13]:
transform=image_df["affine_transform"][0]

In [22]:
kwargs = {}
kwargs.update({
        'transform': transform,
        'driver': "GTIFF",
        'dtype' : "uint8",
        'count': 3,
        'width':
        'height':
    })

In [ ]:
driver

width, height

count

dtype

crs

transform

In [27]:
dst = np.zeros_like(png_arr)

In [31]:
from rasterio.warp import reproject
from rasterio.enums import Resampling
reproject(
        source=png_arr,
        src_transform = transform,
        destination=dst,
        resampling=Resampling.bilinear)

CRSError: Missing src_crs.

In [23]:
import rasterio
with rasterio.open('outputs/RGB.tif', 'w', **kwargs) as dst: #need to define kwargs
    reproject(
        source=png_arr,
        src_transform = transform,
        destination=dst,
        resampling=Resampling.bilinear)

TypeError: Integer width and height are required.

In [ ]:


datum_arr = ski.io.imread(datum_im_p)
ski.io.imshow(datum_arr)

png_arr = ski.io.imread(image_df.png[0])

ski.io.imshow(png_arr)